In [34]:
!pip install openai

In [45]:
import pandas as pd
import numpy as np
import openai
import time
from datetime import datetime

In [46]:
# Load your OpenAI API key
openai_api_key= 'MyAPIKey'  # <- Paste your OpenAI key here

In [64]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model="gpt-3.5-turbo",
# )

def dereference_pronouns_with_openai(text):
    """
    Function to use the OpenAI API for dereferencing pronouns.
    """

    print ("INPUT: ", text)
    # Use the chat model structure for the API call
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant that dereferences pronouns in text."},
            {"role": "user", "content": f"Please return only this same piece of text, but using context, replace all of the pronouns (including  'it', 'its', 'they', 'them', etc) in the text with the entities that are being referenced. : '{text}'"}
        ]
    )
    output_text= response.choices[0].message.content.strip()
    print ("OUTPUT: ", output_text)
    print ()
    time.sleep(1) # Put in a 1 second delay to not overload the OpenAI servers?
    return output_text


In [65]:
# Define a function to apply the dereferencing and handle errors
def robust_dereference(row):
    try:
        # If the text is already processed, skip
        if pd.notna(row['Modified_Text']):
            return row['Modified_Text']
        # Process the text
        modified_text = dereference_pronouns_with_openai(row['Text'])
        row['Modified_Text'] = modified_text
        return modified_text
    except Exception as e:
        print(f"Error processing row {row.name}: {e}")
        time.sleep(10)  # Add a delay before retrying
        return None


In [66]:
# Load the CSV file into a dataframe (adjust the path as necessary)
#df = pd.read_csv("/path_to_your_file/Your_CSV_File.csv")
def add_movie_descriptor(data: pd.DataFrame, corpus_df: pd.DataFrame):
    """
    Adds "Movie Description" to the supplied dataframe, in the form {Genre}_{P|N}_{Movie Title}_{DocID}
    """
    review = np.where(corpus_df['Review Type (pos or neg)'] == 'Positive', 'P', 'N')
    data['Descriptor'] = corpus_df['Genre of Movie'] + '_' + corpus_df['Movie Title'] + '_' + review + '_' + corpus_df['Doc_ID'].astype(str)

def get_corpus_df(path):
    data = pd.read_csv(path, encoding="utf-8")
    add_movie_descriptor(data, data)
    sorted_data = data.sort_values(['Descriptor'])
    indexed_data = sorted_data.set_index(['Doc_ID'])
    indexed_data['Doc_ID'] = indexed_data.index
    return indexed_data
CORPUS_PATH=\
'https://raw.githubusercontent.com/barrycforever/MSDS_453_Public/main/MSDS453_ClassCorpus/MSDS453_ClassCorpus_Final_Sec57_v2_20230928.csv'
class_corpus = get_corpus_df(CORPUS_PATH)
df = class_corpus[class_corpus['Movie Title'] == 'EQUILIBRIUM']

# QA dataset listed above, but examples below are from the class corpus of a particular quarter

In [67]:
# # Don't experiment with all 200 rows of the dataframe the first time!
# df = df.head(3)

In [68]:
df

,DSI_Title,Text,Submission File Name,Student Name,Genre of Movie,Review Type (pos or neg),Movie Title,Descriptor,Doc_ID
Doc_ID,,,,,,,,,
152,BYM_Doc3_EQUILIBRIUM,"A tedious rip-off sci-fi film , itching to be ...",BYM_Doc3_EQUILIBRIUM,BYM,Sci-Fi,Negative,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_N_152,152
155,BYM_Doc6_EQUILIBRIUM,`` Equilibrium '' is like a remake of `` 1984 ...,BYM_Doc6_EQUILIBRIUM,BYM,Sci-Fi,Negative,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_N_155,155
156,BYM_Doc7_EQUILIBRIUM,An unsophisticated sci-fi drama that takes its...,BYM_Doc7_EQUILIBRIUM,BYM,Sci-Fi,Negative,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_N_156,156
157,BYM_Doc8_EQUILIBRIUM,Science fiction is a good genre for creating v...,BYM_Doc8_EQUILIBRIUM,BYM,Sci-Fi,Negative,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_N_157,157
158,BYM_Doc9_EQUILIBRIUM,The worst theatrically released sci-fi flick s...,BYM_Doc9_EQUILIBRIUM,BYM,Sci-Fi,Negative,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_N_158,158
150,BYM_Doc1_EQUILIBRIUM,Equilibrium is my favorite movie of 2002 . It ...,BYM_Doc1_EQUILIBRIUM,BYM,Sci-Fi,Positive,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_P_150,150
151,BYM_Doc2_EQUILIBRIUM,If I was to explain this movie to someone in o...,BYM_Doc2_EQUILIBRIUM,BYM,Sci-Fi,Positive,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_P_151,151
153,BYM_Doc4_EQUILIBRIUM,If you were disappointed by recent sci-fi acti...,BYM_Doc4_EQUILIBRIUM,BYM,Sci-Fi,Positive,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_P_153,153
154,BYM_Doc5_EQUILIBRIUM,`` Equilibrium '' would be a mindless action p...,BYM_Doc5_EQUILIBRIUM,BYM,Sci-Fi,Positive,EQUILIBRIUM,Sci-Fi_EQUILIBRIUM_P_154,154


In [69]:
# Create a new column for modified text
df['Modified_Text'] = None

<ipython-input-69-c45b5e70b37d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Modified_Text'] = None


In [70]:
print ("Start:", datetime.now())
print ()

# Apply the function to the dataframe
df['Modified_Text'] = df.apply(robust_dereference, axis=1)

print ()
print ("Stop:", datetime.now())

Start: 2023-11-26 19:07:09.181419

INPUT:  A tedious rip-off sci-fi film , itching to be another Matrix ( or , maybe just a bearable flick ! ) , that brazenly steals everything from Orwell 's 1984 novel and many other thinking man 's films , with the same Dystopian agenda , such as Fahrenheit 451 . It offers for the viewers of the more low rent sci-fi films the required martial-arts fight sequences and plenty of metallic techno gadgetry ; and , it appeals to a more arty audience by its cleverly created totalitarian gothic set design of mixing actual Berlin locations with CGI effects . Nevertheless , it fails overall to arouse interest dramatically , conceptually or inventively . I 've seen this film 's theme played out too often ( and in a more spirited way ) in recent films to find its well-intentioned virtues necessarily welcome or pleasing . It only adds unneeded celluloid to an increasingly tired genre ! Director-writer Kurt Wimmer ( 1995- '' One Man 's Justice '' ) , a screenwrite

<ipython-input-70-ed8358e00551>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Modified_Text'] = df.apply(robust_dereference, axis=1)


In [ ]:
df['Text'] = df['Modified_Text']
df.drop(columns=['Modified_Text'], inplace=True)

In [ ]:
df

In [71]:
# Save the modified dataframe to a new CSV file
# df.to_csv("/path_to_save/Modified_Movie_Reviews.csv", index=False)
df.to_csv("Dereferenced_Assignment3.csv", index=False)